In [ ]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yaml;

In [ ]:
def load_config(path="../config/config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()
db = config['database']
conn = psycopg2.connect(
    dbname=db['dbname'],
    user=db['user'],
    password=db['password'],
    host=db['host'],
    port=db['port']
)

In [21]:
sql_query = """
With acceptable_lauas as (
    SELECT
    oslaua,
    AVG(park_score) AS avg_park_score
    FROM postcodes
    WHERE park_score IS NOT NULL and park_score > 0.4
    GROUP BY oslaua
    ORDER BY avg_park_score DESC
)
SELECT * from acceptable_lauas
"""

df = pd.read_sql(sql_query, conn)

df

/tmp/ipykernel_111222/292950652.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,oslaua,avg_park_score
0,E06000017,0.963719
1,E09000024,0.931045
2,E09000032,0.837848
3,E06000063,0.750833
4,E06000040,0.726745
5,E07000192,0.723950
6,E07000212,0.697582
7,E06000036,0.675392
8,E09000027,0.674178
9,E07000147,0.674054


In [ ]:
sql_query = """
SELECT postcode,
    oslaua,
    lat,
    long,
    park_score,
    AVG(park_score) OVER(PARTITION BY oslaua) AS avg_park_score_by_local_authority
FROM postcodes
WHERE 
    oslaua = 'E07000095' -- Whichever is selected
    AND park_score IS NOT NULL
    AND park_score > 0.4
ORDER BY park_score DESC;
"""

df = pd.read_sql(sql_query, conn)

df




/tmp/ipykernel_111222/3740368767.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,postcode,oslaua,lat,long,park_score,avg_park_score_by_local_authority
0,EN10 6AA,E07000095,51.728656,-0.025315,0.613446,0.606523
1,EN8 0SS,E07000095,51.712007,-0.035796,0.628829,0.606523
2,EN8 0ST,E07000095,51.711988,-0.036275,0.628829,0.606523
3,EN8 0SU,E07000095,51.712741,-0.035634,0.620947,0.606523
4,EN8 0SW,E07000095,51.711266,-0.035061,0.628829,0.606523
...,...,...,...,...,...,...
2100,EN9 3ZS,E07000095,51.685117,-0.031434,0.595947,0.606523
2101,EN9 3ZT,E07000095,51.685208,-0.031517,0.595947,0.606523
2102,EN9 3ZU,E07000095,51.685117,-0.031434,0.595947,0.606523
2103,EN9 3ZW,E07000095,51.685117,-0.031434,0.595947,0.606523


We could actually return all the postcodes, even though in this case there are 2105 for this area, and display them on a map.
With other criteria we could limit this to maybe the 10 that fit best